In [ ]:
%matplotlib widget
from collections import Counter
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import numpy as np
from rust_ephem import (
    EarthLimbConstraint,
    MoonConstraint,
    SunConstraint,
    TLEEphemeris,
)
from tqdm.notebook import tqdm

from conops.common import ACSMode, unixtime2date
from conops.config import (
    Antenna,
    AttitudeControlSystem,
    Battery,
    Config,
    Constraint,
    FaultManagement,
    FaultThreshold,
    GroundStation,
    GroundStationRegistry,
    Heater,
    Instrument,
    Payload,
    PowerDraw,
    SolarPanel,
    SolarPanelSet,
    SpacecraftBus,
)
from conops.ditl import QueueDITL
from conops.targets import Pointing, Queue

## DITL Configuration


### Read in TLE


In [ ]:
tle_file = "example.tle"

### Set time period to run DITL over

In this example, we'll start 2 days after the epoch of the TLE, at 00:00UT


In [ ]:
length = 1
offset = 2
begin = datetime(2025, 11, 1)
end = begin + timedelta(days=1)

### Generate an Ephemeris for the time period

Using the TLE and the time period defined above, we generate an ephemeris for the spacecraft. This ephemeris will contain the position and velocity of the spacecraft for the entire simulation period, which is essential for determining what the spacecraft can see and when.


In [ ]:
eph = TLEEphemeris(
    begin=begin,
    end=end,
    tle="example.tle",
    step_size=60,
)

### Configure the SpaceCraft Model for the DITL

#### Configure the Spacacraft Bus

The next cell configures the spacecraft bus, which includes defining its power draw in different modes and the parameters for the attitude control system (ACS), such as slew rates and settle times.

##### Heater Power

Here we configure any Spacecraft Bus heaters. These are currently modelled
simply as a fixed power draw, with an (optional) power draw in eclipse.

In [ ]:
heater = Heater(
    name="Bus Heaters",
    power_draw=PowerDraw(nominal_power=25, eclipse_power=50),
)

##### Attitude Control System

Define Attitude Control System Parameters. Here we define the characteristics
of the ACS system, right now this is just to determine how fast the spacecraft slews.

In [ ]:
# Define the parameters for the attitude control system, currently this is
# defining the slew rate and acceleration limits, as well as the settle time after
# a slew.
acs = AttitudeControlSystem(
    settle_time=10,  # Units of seconds
    max_slew_rate=0.3,  # Units of deg/s
    slew_acceleration=0.01,  # Units of deg/s^2
)

#### Slew time sanity check

The follow plot shows the calculated slew time as a function of slew degrees,
this acts as a good sanity check against putting in bad numbers that can break
the simulation.


In [ ]:
times = np.arange(0, 180)
plt.figure()
plt.plot(times, [acs.slew_time(t) for t in times])
plt.xlabel("Slew Angle (deg)")
plt.ylabel("Slew Time (s)")
plt.title("Slew Time vs. Slew Angle")

##### Spacecraft Bus power

This defines the power draw of the spacecraft bus, depending on ACS mode (e.g.
slewing, safe mode etc).

In [ ]:
# Configure the power draw for the spacecraft bus in various
# In this case, the spacecraft bus draws 70W in science mode, 100W while
# slewing, and has a nominal power draw of 50W in all other ACS modes.
power_draw = PowerDraw(
    nominal_power=50,
    power_mode={ACSMode.SCIENCE: 70, ACSMode.SLEWING: 100},
)

##### Final Spacecraft Bus configuration

As Heaters and ACS are subsystems of the Spacecraft Bus, they are given as
arguments to the overall Spacecraft Bus Configuration

In [ ]:
# Configure the spacecraft bus with the power draw and ACS defined above.
spacecraft_bus = SpacecraftBus(
    name="Example Spacecraft Bus",
    power_draw=power_draw,
    attitude_control=acs,
    heater=heater,
)

#### Configure the Instruments

Configure the instruments. Note a spacecraft can have multiple instruments, so
the `Payload` class holds all the instruments onboard. Right now a
Instrument is just a thing that draws power. The `PowerDraw` class defines how
much power the instrument draws in various ACS modes.


In [ ]:
instrument_power = PowerDraw(
    nominal_power=50,
    power_mode={ACSMode.SCIENCE: 200, ACSMode.SLEWING: 50, ACSMode.SAA: 50},
)
optical_instrument = Instrument(
    name="Optical Imager",
    power_draw=instrument_power,
)

payload = Payload(payload=[optical_instrument])

#### Configure the Battery

Define the capacity and maximum allowed depth of discharge in nominal operations.


In [ ]:
battery = Battery(watthour=1000, max_depth_of_discharge=0.4)

#### Configure the Solar Panel

Define the physical configuration of the solar panel, the max power generation
and the conversion efficiency.


In [ ]:
# Configure each solar panel on the spacecraft, in this case we only have one
# fixed side mounted panel.
solar_panel = SolarPanel(
    name="Example Solar Panel",
    gimbled=False,  # not gimbled, if gimbled then would track sun
    sidemount=True,  # panel mounted on side of spacecraft, as opposed to pointing opposite of pointing direction
    max_power=474.21,  # Max power generation in Watts
    conversion_efficiency=0.94,  # Efficiency of solar panel
)
# Create a solar panel set to hold all the panels on the spacecraft
solar_panel_set = SolarPanelSet(panels=[solar_panel])

#### Configure the observing constraints

Observing constraints are defined using `rust-ephem` module's constraints,
basic observing constraints are Sun, Moona and Earth Limb avoidance. CONOPS
simulator has a built in Solar Panel Constraint that allows for the minimum
incidence angle of the Sun on the solar panel to be a constraint, not that
during spacecraft eclipse, this angle is not enforced.

Note here that the solar panel constraint is set up so that the Sun has to
be within 45 degrees of the solar panel. This is defined using a SunConstraint
where targets < 45 degrees from the Sun and < 45 degrees from the anti-Sun are
not available.


In [ ]:
# We define here the maximum solar panel angle for the solar panel constraint
max_solar_panel_angle = 45  # degrees


constraint = Constraint(
    ephem=eph,
    sun_constraint=SunConstraint(min_angle=90),  # & ~EclipseConstraint(),
    moon_constraint=MoonConstraint(min_angle=20),
    earth_constraint=EarthLimbConstraint(min_angle=20),
    panel_constraint=SunConstraint(
        min_angle=90 - max_solar_panel_angle, max_angle=90 + max_solar_panel_angle
    ),
)

#### Ground Station Configuration

We can define the locations of ground stations, and their capabilities. Here I
made up some ground stations for the sims. Note that each ground station comes
with a `schedule_probability` parameter. This is in order to simulate network
congestion, and to randomize the DITL. Essentially every time the groundstation
is in view, we roll a dice to determine if this one is scheduled. Schedule
probability = 1 means every time.


In [ ]:
# Define a custom set of ground stations
# This is an example, you can add or remove stations as needed.
# The default ground stations are used in this example.

nairobi = GroundStation(
    code="NBO",
    name="Nairobi",
    latitude_deg=-1.2921,
    longitude_deg=36.8219,
    elevation_m=0.0,
    min_elevation_deg=10.0,
    antenna=Antenna(bands=["S"]),
    schedule_probability=0.7,
)

south_pole = GroundStation(
    code="SPO",
    name="South Pole",
    latitude_deg=-90.0,
    longitude_deg=0.0,
    elevation_m=2835.0,
    min_elevation_deg=10.0,
    antenna=Antenna(bands=["S"]),
    schedule_probability=0.2,
)

north_pole = GroundStation(
    code="NPO",
    name="North Pole",
    latitude_deg=90.0,
    longitude_deg=0.0,
    elevation_m=0.0,
    min_elevation_deg=10.0,
    antenna=Antenna(bands=["S"]),
    schedule_probability=0.2,
)

punta_veija = GroundStation(
    code="PVG",
    name="Punta Veija",
    latitude_deg=8.7832,
    longitude_deg=-71.2325,
    elevation_m=0.0,
    min_elevation_deg=10.0,
    antenna=Antenna(bands=["S"]),
    schedule_probability=0.5,
)


# Create a GroundStationRegistry with the custom stations
ground_station_registry = GroundStationRegistry(
    stations=[nairobi, south_pole, north_pole, punta_veija]
)

In [ ]:
fault_management = FaultManagement(
    thresholds={
        "battery_level": FaultThreshold(
            name="battery_level", yellow=0.5, red=0.3, direction="below"
        )
    }
)

#### Create the configuration object

This configuration object holds all the spacecraft subsystems and constraints.
It is a Pydantic model, so this can be easily serialized to JSON so the
configuration can be saved.


In [ ]:
# Create the configuration object


config = Config(
    name="Example Spacecraft Configuration",
    spacecraft_bus=spacecraft_bus,
    solar_panel=solar_panel_set,
    payload=payload,
    battery=battery,
    constraint=constraint,
    ground_stations=ground_station_registry,
    fault_management=fault_management,
)

#### Save the configuration

Configuration can be saved to JSON for writing to disk. This way configuration
steps above can be done only once, and the JSON file can be part of a fixed,
version controlled spacecraft configuration.


In [ ]:
# Serialize to JSON and save to disk
config.to_json_file("example_config.json")

# Read JSON from disk
config = Config.from_json_file("example_config.json")

# Note that the ephemeris is not serialized, so we need to set it again
config.constraint.ephem = eph

### Ingest Targets for the DITL simulation

In this step we ingest targets into our simulation. This could be a list of
real science targets, an astronomical catalogue. For this simple example, we'll
just generate a list of random points on the sky.


In [ ]:
number_of_targets = 1000
target_ra, target_dec = (
    np.random.uniform(0, 360, number_of_targets),
    np.random.uniform(-90, 90, number_of_targets),
)
print(f"Number of pointings = {len(target_ra)}")

#### Populate Target Queue

Take the list of targets, and use them to populate the Target `Queue`. This also
pre-calculates the visibility windows for each target.


In [ ]:
targids = list(range(10000, 10000 + len(target_ra)))

targlist = Queue()
targlist.ephem = eph
for i in tqdm(range(len(targids))):
    pointing = Pointing(
        constraint=config.constraint, acs_config=config.spacecraft_bus.attitude_control
    )
    pointing.merit = 40
    pointing.ra = target_ra[i]
    pointing.dec = target_dec[i]
    pointing.obsid = targids[i]
    pointing.name = f"pointing_{pointing.obsid}"
    pointing.exptime = 5000
    pointing.visibility()
    targlist.append(pointing)

### Set up the Queue Scheduled DITL


In [ ]:
# %%prun
ditls = list()
for i in range(1):
    targlist.reset()
    ditl = QueueDITL(config=config)
    ditl.acs.last_slew = None
    ditl.queue = targlist
    ditl.ephem = eph
    ditl.begin = begin
    ditl.end = end
    ditl.calc()
    ditls.append(ditl)

#### Check to see if any Battery charging events happened


In [ ]:
# Check for emergency charging behavior
charging_cmds = [
    cmd
    for cmd in ditl.acs.executed_commands
    if "BATTERY_CHARGE" in cmd.command_type.name
]
print(f"Total battery charge commands: {len(charging_cmds)}")
for i, cmd in enumerate(charging_cmds[:20]):  # Show first 20
    print(f"{i}: {unixtime2date(cmd.execution_time)}: {cmd.command_type.name}")

In [ ]:
# Convert your data
times = [unixtime2date(t) for t in ditl.utime]
modes = [m.name for m in ditl.mode]
battery_levels = ditl.batterylevel  # unused in this example, but you can use later

# Count occurrences of each mode
mode_counts = Counter(modes)

# Prepare data for pie chart
labels = list(mode_counts.keys())
sizes = list(mode_counts.values())

# Create pie chart
plt.figure(figsize=(10, 8))
plt.pie(sizes, labels=labels, autopct="%1.1f%%", startangle=140)
plt.title("Percentage of Time Spent in Each ACS Mode")
plt.axis("equal")  # Equal aspect ratio ensures pie is a circle
plt.show()

#### Plot the output of the DITL simulation

This plot shows spacecraft RA/Dec over time, ACS mode, Battery Charge, Solar
Panel illumination, power level and observation ID


In [ ]:
# Plot DITL results
ditl.plot()

### Print summary statistics about the DITL sim




In [ ]:
ditl.print_statistics()